# Homework 4

Apply a sequence of cuts to the data to identify the decay $K_S \rightarrow \pi^0 \pi^0$:
* Evaluate the number $N(K_S \rightarrow \pi^0 \pi^0)$ of signal events after the sequence.
* Provide a table with the cutflow: List of each cut and the number of candidate events surviving after each cut.

Apply a sequence of cuts to the data to identify the decay $K_S \rightarrow \pi^+ \pi^-$:
* Evaluate the number $N(K_S \rightarrow \pi^+ \pi^-)$ of signal events after the sequence.
* Provide a table with the cutflow: List of each cut and the number of candidate events surviving after each cut.

Evaluate the ratio $$\frac{N(K_S \rightarrow \pi^+ \pi^-)}{N(K_S \rightarrow \pi^0 \pi^0)}$$

## Data

The data will be acquired, and the data from each event will be grouped into arrays, which will then be stored in an array of arrays.

In [2]:
# Libraries
import numpy as np  

In [3]:
def process_dat(data):
    
    events = []  # List to store matrices
    current_event = []  # Current event being constructed

    
    with open(data, 'r') as data:
        for line in data:
            # Split line and convert values to numbers
            values = line.split()
            if len(values) < 1:  # Skip empty lines
                continue
            
            values = [float(val) for i, val in enumerate(values)]
            
            # Check if the first column contains zero
            if values[0] == 0:
                if current_event:
                    events.append(np.array(current_event))  # Save current matrix
                current_event = []  # Start a new matrix    
            current_event.append(values)  # Append the entire line
            
        # Add the last matrix
        if current_event:
            events.append(np.array(current_event))
    
    return events

# Example usage:
data = 'kloe.dat'
events = process_dat(data)

print("The total number of events is: ", len(events))

# # Access the events:
# for idx, event in enumerate(events):
#     print(f"Matrix {idx+1}:")
#     print(event)

The total number of events is:  254327


In [4]:
def energy_resolution(E):
    """
    Calculate energy resolution with cluster energy [MeV].

    Args:
        E (float): Energy [MeV].
    Returns:
        float: Energy resolution.
    """
    E = E*10**(-3)
    return 0.057*np.sqrt(E)

def time_resolution(E):
    """
    Calculate time resolution with cluster energy [MeV].

    Args:
        E (float): Energy [MeV].
    Returns:
        float: Time resolution [s].
    """
    return (np.sqrt((54 / np.sqrt(E*10**(-3)))**2 + 100**2))*10**(-12)  # Total time resolution in s


The total number of events before cuts is $254327$

## $K_S \rightarrow \pi^0 \pi^0$

The cuts to be made in this selection will be

* **Clusters:** Events with at least 5 clusters, $4 \gamma$, and $1 K_L$ are selected.
* **Energy:** The energy of the clusters in the calorimeter must be $20\text{ MeV} < E_{cl} < 300 \text{ MeV}$
* **Polar angle:** The polar angle should be $\theta = \arccos{\frac{z}{R}} > 21 \degree$, with $R = \sqrt{x^2 + y^2 + z^2}$
* **Photons:** Selection of events with at least $4$ photons, which is ensured by the following condition.
  $$
    t_{\text{cluster}} - \frac{d_{\text{cluster}}}{c} \approx 0
  $$
  $t_{\text{cluster}}$ is the cluster time, $d_{\text{cluster}}$ is the cluster's distance from the interaction point and $c$ is the speed of light.
* **$K_L$ Crash:** Identification of a cluster with energy $E > 150$ MeV, such that,
  $$
    t_{\text{cluster}} - \frac{d_{\text{cluster}}}{\beta_{K_L}} \approx 0
  $$
  with
  $$
    \beta_{K_L} = \frac{p_{K_L}}{E_{\text{cluster}}}c
  $$

  $$
    \beta_{K_L} \approx \frac{1}{5} p_{\phi}
  $$



In [5]:
# Clusters cuts

def clusters_cut(events):
    cluster_cut = []
    for event in events:
        count = np.sum(event[:, 0] == 1)
        if count >= 5:
            cluster_cut.append(event)
    return cluster_cut

clust_cut = clusters_cut(events)

print(f"Number of surviving events: {len(clust_cut)}")

# # Access the surviving events:
# for idx, event in enumerate(clust_cut):
#     print(f"Surviving Event {idx+1}:")
#     print(event)

Number of surviving events: 168200


In [6]:
# # Energy cut

# def energy_cut(events):

#     energy_events = []
#     for event in events:
#         count = 0
#         for row in event:
#             if row[0] == 1 and (20 - energy_resolution(row[6])) <= row[6] <= (300 + energy_resolution(row[6])):
#                 count += 1
#         if count >= 5:
#             energy_events.append(event)
#     return energy_events

# cut_energy = energy_cut(clust_cut)

# # Print and save surviving matrices
# num_energy_cut = len(cut_energy)
# print(f"Number of events survived after the energy cut: {num_energy_cut}")

In [7]:
# Energy cut

def energy(events):

    energy_events = []
    for event in events:
        valid_event = True
        for row in event:
            if row[0] == 1 and not (20 <= row[6] <= 300):
                valid_event = False
                break
        if valid_event:
            energy_events.append(event)
    return energy_events

cut_energy1 = energy(clust_cut)
# cut_energy1 = energy(events)

# Print and save surviving matrices
num_energy_cut1 = len(cut_energy1)
print(f"Number of events survived after the energy cut: {num_energy_cut1}")

Number of events survived after the energy cut: 18135


In [8]:
# Polar angle cut

def polar_cut(events):
    angle_treshold = 20
    angle_resolution = 1
    polar_events = []
    for event in events:
        count = 0
        for row in event:
            if row[0] == 1:
                # Calculate magnitude R of cluster (x, y, z)
                R = np.sqrt(row[2]**2 + row[3]**2 + row[4]**2)
                # Calculate angle with z-axis in degrees
                angle = np.degrees(np.arccos(np.abs(row[4] / R)))
                if angle >= (angle_treshold - angle_resolution):
                    count += 1
        if count >= 5:
            polar_events.append(event)
    return polar_events

# Filter polar events based on angle with z-axis condition
polar_events = polar_cut(cut_energy1)

# Print and save surviving matrices
num_survived_polar_events = len(polar_events)
print(f"Number of matrices left in polar_events: {num_survived_polar_events}")

Number of matrices left in polar_events: 16551


In [9]:
# # Polar angle cut

# def polar_cut(events):
#     angle_treshold = 20
#     angle_resolution = 1
#     polar_events = []
#     for event in events:
#         valid_event = True
#         for row in event:
#             if row[0] == 1:
#                 # Calculate magnitude R of cluster (x, y, z)
#                 R = np.sqrt(row[2]**2 + row[3]**2 + row[4]**2)
#                 # Calculate angle with z-axis in degrees
#                 angle = np.degrees(np.arccos(np.abs(row[4] / R)))
#                 if angle <= (angle_treshold - angle_resolution):
#                     valid_event = False
#                     break
#         if valid_event:
#             polar_events.append(event)
#     return polar_events

# # Filter polar events based on angle with z-axis condition
# polar_events = polar_cut(cut_energy1)

# # Print and save surviving matrices
# num_survived_polar_events = len(polar_events)
# print(f"Number of matrices left in polar_events: {num_survived_polar_events}")

In [10]:
# Photons cut

def find_distance(point):
    # Calculate Euclidean distance between cluster and ineraction point
    return np.sqrt(np.sum((point) ** 2))

def photons_cut(events):
    photons = []
    c = 29.9792458  # speed of light in cm/ns
    for event in events:
        clusters = []
        for row in event:
            if row[0] == 1:
                clusters.append(row[2:7])  # Extract cluster coordinates, time, and energy

        if len(clusters) < 4:
            continue

        # Calculate distances between origin and clusters
        cluster_distances = [find_distance(cluster[:3]) for cluster in clusters]

        # Check condition for at least 4 clusters
        count = 0
        for i, cluster_dist in enumerate(cluster_distances):
            cluster_time = clusters[i][3]
            cluster_energy = clusters[i][4]
            delta_t = cluster_time - cluster_dist / c
            # if -100 * time_resolution(cluster_energy) <= delta_t <= 100 * time_resolution(cluster_energy):
            if 0 <= delta_t <= 0.9: # Time resolution 0.1 ns
                count += 1

        if count >= 4:
            photons.append(event)

    return photons

four_photons = photons_cut(polar_events)

# Print and save surviving events
num_survived_four_photons = len(four_photons)
print(f"Number of events left in four_photons: {num_survived_four_photons}")


Number of events left in four_photons: 1781


In [11]:
### K long energy cut

def KL_cut(events):
    crash = []
    c = 29.9792458  # speed of light in cm/ns
    for event in events:
        interaction_point_momentum = None
        for row in event:
            if row[0] == 0:
                p_momentum = row[5:8]  # Extract interaction point momentum
                break
                
        # Calculate momentum magnitude of phi
        phi_momentum = find_distance(p_momentum) * c

       # Calculate cluster speed using the momentum of the interaction point
        for row in event:
            if row[0] == 1:
                cluster_energy = row[6]
                if cluster_energy >= 150: #MeV
                    # Calculate cluster speed
                    cluster_momenum = 0.2 * phi_momentum
                    cluster_speed = cluster_momenum * c / cluster_energy
                    cluster_time = row[5]
                    cluster_mag = find_distance(np.array(row[2:5]))
                    delta_t = cluster_time - cluster_mag / cluster_speed
                    if 0 <= delta_t <= 0.9:
                        crash.append(event)
                        break

    return crash

KL_crash = KL_cut(four_photons)

# Print and save surviving events
num_survived_KL_crash = len(KL_crash)
print(f"Number of events left in KL crash: {num_survived_KL_crash}")

Number of events left in KL crash: 15


### List of cuts

| Cut Name    | Before |  After |
| --------    | ------ | ------ |
| Clusters    | 254327 | 168200 |
| Energy      | 168200 | 18135  |
| Polar Angle | 18135  | 16551  |
|Photons      |16551   | 1781   |
|$K_K$ Crash  |1781    | 15     |

## $K_L \rightarrow \pi^+ \pi^-$

Now, 3 cuts will be made on the original events.

* **Vertex and cluster:** Data with at least one vertex and one cluster will be selected.
* **Vertex IP** A vertex very close to the interaction point is selected.
  $$
    |z(vxt)| < 5 \text{ cm}
  $$

  $$
    |R_{\perp(vxt)}| = \sqrt{x^2_{vxt} + y^2_{vxt}} < 4 \text{ cm}
  $$
* **Kinematics** We start by selecting vertices where the sum of the momenta of two tracks, originating from the vertex, aligns with the expected momentum of the K_S. Next, we reconstruct the momentum expected for K_L in a two-body decay, allowing us to determine the beta of the K_L. Finally, we tag the K_L by examining its time of flight, ensuring that its cluster has an energy greater than 150MeV.

In [12]:
# Initialize list to store selected events
events_clus_vxt = []

# Iterate over each event in the events
for event in events:
    # Check if both 1 and 2 are present in the first column
    if any(row[0] == 1 for row in event) and any(row[0] == 2 for row in event):
        events_clus_vxt.append(event)

# Print the number of selected matrices
print(f"Number of selected events: {len(events_clus_vxt)}")

Number of selected events: 195907


In [13]:
def vxtIP(events):
    selected_events = []
    for event in events:
        for row in event:
            if row[0] == 2:
                vertex_z = abs(row[4])  # Absolute value of z coordinate of the vertex
                vertex_xy_magnitude = find_distance(np.array(row[2:4]))  # Magnitude of vector in xy plane
                if -5 < vertex_z < 5 and vertex_xy_magnitude < 4:
                    selected_events.append(event)
                    break  # Once a vertex meets the condition, no need to check other vertices in the same matrix
    return selected_events

vxtIP_cut = vxtIP(events_clus_vxt)

# Print and save surviving events
num_survived_vxtIP_cut = len(vxtIP_cut)
print(f"Number of events left in KL crash: {num_survived_vxtIP_cut}")

Number of events left in KL crash: 116537


In [14]:
def Kin(events):
    c = 29.9792458  # speed of light in cm/ns
    K_m = 497.611
    kinematics = []
    for event in events:
        # phi_momentum = np.sqrt(event[0][5]**2 + event[0][6]**2 + event[0][7]**2) * c #MeV
        phi_momentum = find_distance(np.array(event[0][5:8])) * c #MeV
        for row in event:
            if row[0] == 2:
                vtx_px = row[5] + row[8]
                vtx_py = row[6] + row[9]
                vtx_pz = row[7] + row [10]
                vtx_p = [vtx_px, vtx_py, vtx_pz]
                # Ks_p = np.sqrt( px_vtx**2 + py_vtx**2 + pz_vtx**2 )
                Ks_p = find_distance(np.array(vtx_p))
                
                
                K_E = event[0][8]/2 # sqrt(s) / 2 (2 dacay)
                K_p = np.sqrt(K_E ** 2 - K_m ** 2) # Expected momentum of K
                if 0 < abs(K_p - Ks_p) < 2.5:
                    Kl_p = np.sqrt((vtx_p[0] - event[0][5]) ** 2 +
                                   (vtx_p[1] - event[0][6]) ** 2 +
                                   (vtx_p[2] - event[0][7]) ** 2)
                    beta = Kl_p / np.sqrt(Kl_p ** 2 + K_m ** 2)
                    for row1 in event:
                        if row1[0] == 1 and row1[6] > 150:
                            cluster_speed = Kl_p * c / row1[6]
                            cluster_time = row1[5]
                            cluster_mag = find_distance(np.array(row[2:5])) #cm
                            if 0 <= cluster_time - cluster_mag / (beta * c) <= 0.9:
                                kinematics.append(event)
    return kinematics

kine_cut = Kin(vxtIP_cut)

# Print and save surviving events
num_survived_kine_cut = len(kine_cut)
print(f"Number of events left in KL crash: {num_survived_kine_cut}")

print(f"Ratio charged over neutral: {num_survived_kine_cut / num_survived_KL_crash}")

Number of events left in KL crash: 34
Ratio charged over neutral: 2.2666666666666666


### List of cuts

| Cut Name          | Before |  After |
| ----------------- | ------ | ------ |
| Min clus & vtx    | 254327 | 195907 |
| Vertex pos        | 195907 | 116537 |         
| Kinematics        | 116537 | 34     |

From PDG

* $P(\phi \rightarrow K_L K_S) = 0.34$
* $P(K_S \rightarrow \pi^+ \pi^-) = 0.6920$
* $P(K_S \rightarrow \pi^0 \pi^0) = 0.3069$

Expected ratio

$$
    R^{\text{TH}} = \frac{N(K_S \rightarrow \pi^+ \pi^-)}{N(K_S \rightarrow \pi^0 \pi^0)} = 2.25
$$

Obtained

$$
    R^{\text{Measured}} = \frac{N(K_S \rightarrow \pi^+ \pi^-)}{N(K_S \rightarrow \pi^0 \pi^0)} = 2.27
$$

Percentage difference $0.09\%$